# Block 1: Installtions and drives

block 1: installations and drives

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
your_module = drive.CreateFile({'id':'1YlZBlvViO4Iz0xU-6aPS5lUzL6O0nxHg'})

your_module.GetContentFile('spacy_wrapper.py')

your_module2= drive.CreateFile({'id':'1l8Sd_CrGY8Mk6hy9uhwGZnkrKRwSACX5'})

your_module2.GetContentFile('symbols.py')

your_module3= drive.CreateFile({'id':'1X6gc4BOll6iweDl7gFs4PmPi5w514Qhg'})

your_module3.GetContentFile('load_pretrained_word_embeddings.py')

your_module4= drive.CreateFile({'id':'1XhgGTsqee49it6xdLdAa69cWNVNGNdX5'})

your_module4.GetContentFile('word_index.py')

your_module5= drive.CreateFile({'id':'1NO71gAICxxpe3JfRcvTFicQIhUiHpIPZ'})

your_module5.GetContentFile('elmo.py')

your_module6= drive.CreateFile({'id':'1xUn888UMox08U2dxgBR1WmULPKQ0jyzB'})

your_module6.GetContentFile('utils.py')

your_module7= drive.CreateFile({'id':'1Y3pdrMF-5cha__aG3MKHkj0ydhBO0BDX'})

your_module7.GetContentFile('Alayers.py')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import keras
import os
import sys
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, Activation, Dropout
from typing import List
import torch
import os
import re
torch.cuda.current_device(),torch.cuda.device_count(),torch.cuda.is_available()
!pip install tiny-tokenizer flair
!pip install flair
import nltk
nltk.download('averaged_perceptron_tagger')

from flair.embeddings import XLNetEmbeddings
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.data import  Sentence, Token
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, \
    CharLMEmbeddings
from typing import List
import torch
import os
import re
from flair.embeddings import BertEmbeddings
from flair.embeddings import XLMRobertaEmbeddings



Mounted at /content/gdrive


# Block 2: Creating XLNet Embeddings Function

Block 2: Creating XLNet embeddings

In [ ]:
xln = BertEmbeddings()
emb_size= 2048
# init embedding
xln = XLMRobertaEmbeddings()
emb_size= 1024 
# init embedding
from flair.embeddings import WordEmbeddings
# init embedding
xln = WordEmbeddings('glove')
emb_size= 100

In [ ]:
def create_XLNET_embeddings(elmo, documents, max_sentences = 10):
    embeddings = []
    labels = []
    xln = XLMRobertaEmbeddings()
    emb_size= 1024 
    while True:
      for row in documents:
          my_sent = row
          sentence = Sentence(str(my_sent))
          xln.embed(sentence)
          
          x = []
          for token in sentence:
            x.append(token.embedding.cpu().detach().numpy())
            if len(x) == max_sentences:
              break
          
          while len(x) < max_sentences:
            x.append(np.zeros(emb_size))          
          embeddings.append(x)            
      return np.array(embeddings)
    
    return embeddings        

# Block 3: All functions + Hyperparameters

In [1]:
#-----------Disclaimer: Most of the functions here are originally from with some modifications --> Stanvosky et al. Please refer/cite his work: Stanovsky, Gabriel, et al. "Supervised open information extraction." Proceedings of the 2018 Conference 
#of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long Papers). 2018. ---------------
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.utils import np_utils
import pandas as pd


sent_maxlen = 20
classes = None
encoder =  LabelEncoder()
batch_size = 5
epochs=10
emb_dropout = 0.1
trainable_emb = True
pos_tag_embedding_size = 5
num_of_latent_layers = 2
hidden_units = pow(2, 7)
pred_dropout=0.1 
class Sample:
    """
    Single sample representation.
    Containter which names spans in the input vector to simplify access
    """
    def __init__ (self, word):
        self.word = word

    def encode(self):
        """
        Encode this sample as vector as input for rnn,
        Probably just concatenating members in the right order.
        """
        return self.word
      
class Pad_sample(Sample):
    """
    A dummy sample used for padding
    """
    #Sample( word = 0 )
    def __init__(self):
        Sample.__init__(self, word = 0)    
        
def pad_sequences(sequences, pad_func, maxlen = None):
    """
    Similar to keras.preprocessing.sequence.pad_sequence but using Sample as higher level
    abstraction.
    pad_func is a pad class generator.
    """
    ret = []

    max_value = max(map(len, sequences))
    if maxlen is None:
        maxlen = max_value

    # Pad / truncate (done this way to deal with np.array)
    for sequence in sequences:
        cur_seq = list(sequence[:maxlen])
        cur_seq.extend([pad_func()] * (maxlen - len(sequence)))
        ret.append(cur_seq)
    return ret


def classes_():    
        """
        Return the classes which are classified by this model
        """
        return encoder.classes_     

def num_of_classes():
        """
        Return the number of ouput classes
        """
        return len(classes_())
        
def load_dataset(fn,T):
      
      df = pandas.read_csv(fn,sep= "\t",header=0,keep_default_na=False)

      if T=="train":
        encoder.fit(df.label.values)
      else: 
            df = pandas.read_csv(fn,
                         sep= "\t",
                         header=0,
                         keep_default_na=False)
      sents = get_sents_from_df(df)
           
      return (encode_outputs(sents))  
    
def load_datasetTestRE(fn,T):
    
      if T=="test_RE":
          df = pandas.read_csv(fn,
                         sep= ";",
                         header=0,
                         keep_default_na=False)
          df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
          df.word_id = pd.to_numeric(df.word_id, errors='coerce').astype('Int64')
          df.run_id = pd.to_numeric(df.run_id, errors='coerce').astype('Int64')
          df.sent_id = pd.to_numeric(df.sent_id, errors='coerce').astype('Int64')
          df.head_pred_id = pd.to_numeric(df.head_pred_id, errors='coerce').astype('Int64')
          df = df[df.word.apply(lambda x : len(x)>0)] 
      else:
            df = pandas.read_csv(fn,
                         sep= "\t",
                         header=0,
                         keep_default_na=False)
     
      if T=="train":
        encoder.fit(df.label.values)
        
      sents = get_sents_from_df(df)    
      return (encode_outputs(sents))
  
def load_dataset_encodeinputs(fn,T):
      if "RE" in T:
          df = pandas.read_csv(fn,
                         sep= ";",
                         header=0,
                         keep_default_na=False)
          df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
          df.word_id = pd.to_numeric(df.word_id, errors='coerce').astype('Int64')
          df.run_id = pd.to_numeric(df.run_id, errors='coerce').astype('Int64')
          df.sent_id = pd.to_numeric(df.sent_id, errors='coerce').astype('Int64')
          df.head_pred_id = pd.to_numeric(df.head_pred_id, errors='coerce').astype('Int64')
          df = df[df.word.apply(lambda x : len(x)>0)] #to skip null values

      else:
        df = pandas.read_csv(fn,sep= "\t",header=0,keep_default_na=False)

      if T=="train" or T=="POS_train" or T=="train_xlnet_words" or T=="train_xlnet_pred":
        encoder.fit(df.label.values)
      
      
      sents = get_sents_from_df(df)
      if T=="POS_train" or T=="POS" or T== "dev_xlnet_pos" or T=="test_xlnet_pos_RE" or T=="test_xlnet_pos_": 
        return getPOS(sents)
      if  T=="train_xlnet_words" or T=="train_xlnet_pred" or T== "dev_xlnet_words" or T== "dev_xlnet_pred" or T=="test_xlnet_words_RE" or T=="test_xlnet_pred_RE" or T=="test_xlnet_words_" or T=="test_xlnet_pred_"  : #13 july 2020 added this or T=="test_xlnet_words_" or T=="test_xlnet_pred_" :
          return encodeXLnetInputs(sents,T)
      return (encode_inputs(sents))    
    
def get_sents_from_df( df):
      #Split a data frame by rows accroding to the sentences
      return [df[df.run_id == run_id]
            for run_id
            in sorted(set(df.run_id.values))]  
def transform_labels(labels):
        """
        Encode a list of textual labels
        """
        # Fallback:
        classes  = list(classes_())
        return [classes.index(label) for label in labels]

  
def get_fixed_size(sents):
        """
        Partition sents into lists of sent_maxlen elements
        (execept the last in each sentence, which might be shorter)
        """
        return [sent[s_ind : s_ind + sent_maxlen]
                for sent in sents
                for s_ind in range(0, len(sent), sent_maxlen)]      

    
def encode_outputs(sents):
        output_encodings = []
        sents = get_fixed_size(sents)
        # Encode outputs
        for sent in sents:
            output_encodings.append(list(np_utils.to_categorical(list(transform_labels(sent.label.values)),
                                                                 num_classes = num_of_classes())))
        
        # Pad / truncate to maximum length
      
        return np.ndarray(shape = (len(sents),
                                  sent_maxlen,
                                  num_of_classes()),
                          buffer = np.array(pad_sequences(output_encodings,
                                                          lambda : \
                                                            np.zeros(num_of_classes()),
                                                          maxlen = sent_maxlen)))
def get_head_pred_word( full_sent):
       
        """
        Get the head predicate word from a full sentence conll.
        """
        assert(len(set(full_sent.head_pred_id.values)) == 1) # Sanity check
        pred_ind = full_sent.head_pred_id.values[0]
        
        return full_sent.word.values[pred_ind] \
            if pred_ind != -1 \
               else full_sent.pred.values[0].split(" ")[0]
def predict_classes():
        """
        Predict to the number of classes
        Named arguments are passed to the keras function
        """
        return lambda x: stack(x,
                                    [lambda : TimeDistributed(Dense(output_dim = num_of_classes(),
                                                                    activation = "softmax"))] +
                                    [lambda : TimeDistributed(Dense(hidden_units,
                                                                    activation='relu'))] * 3)
        
def encode_inputs(sents):
        word_inputs = []
        pred_inputs = []
        pos_inputs = []
    
        assert(all([len(set(sent.run_id.values)) == 1
                    for sent in sents]))        
         
        run_id_to_pred = dict([(int(sent.run_id.values[0]),
                                get_head_pred_word(sent))
                               for sent in sents])
        # Construct a mapping from running word index to pos
        word_id_to_pos = {}
        for sent in sents:
            indices = sent.index.values
            words = sent.word.values
            
            for index, word in zip(indices,
                                    spacy_ws(" ".join(words))):
                word_id_to_pos[index] = word.tag_  #tag_ get the tag of the word by spacy
        
        
        fixed_size_sents = get_fixed_size(sents)
          

        for sent in fixed_size_sents:

            assert(len(set(sent.run_id.values)) == 1)

            word_indices = sent.index.values
            sent_words = sent.word.values

            
            pos_tags_encodings = [p for (x,p) in nltk.pos_tag(sent_words)]
            word_encodings = [ w for w in sent_words]  
            pred_word = run_id_to_pred[int(sent.run_id.values[0])]
                                    
            pred_word_encodings = [pred_word]
            
            word_inputs.append([Sample(w) for w in word_encodings])
            pred_inputs.append([Sample(w) for w in pred_word_encodings])
            pos_inputs.append([Sample(pos) for pos in pos_tags_encodings])
          
        ret = defaultdict(lambda: [])
         
        for name, sequence in zip(["word_inputs", "predicate_inputs","postags_inputs"],
                                  [word_inputs, pred_inputs,pos_inputs]): 
            for samples in pad_sequences(sequence,
                                         pad_func = lambda : Pad_sample(),
                                         maxlen = sent_maxlen):
                ret[name].append([sample.encode() for sample in samples])
        injy={k: np.array(v) for k, v in ret.items()} 
        return {k: np.array(v) for k, v in ret.items()} 

def getPOS(sents):
        word_inputs = []
        pred_inputs = []
        pos_inputs = []
        assert(all([len(set(sent.run_id.values)) == 1
                    for sent in sents]))        
         
        run_id_to_pred = dict([(int(sent.run_id.values[0]),
                                get_head_pred_word(sent))
                               for sent in sents])
        
        # Construct a mapping from running word index to pos
        word_id_to_pos = {}
        for sent in sents:
            indices = sent.index.values
            words = sent.word.values
       
            for index, word in zip(indices,
                                    spacy_ws(" ".join(words))):
                word_id_to_pos[index] = word.tag_  #tag_ get the tag of the word by spacy
        
        fixed_size_sents = get_fixed_size(sents)
    
        allPosEncoding=[] 

        for sent in fixed_size_sents:

            assert(len(set(sent.run_id.values)) == 1)

            word_indices = sent.index.values
            sent_words = sent.word.values
             
            pos_tags_encodings = [p for (x,p) in nltk.pos_tag(sent_words)]
            word_encodings = [ w for w in sent_words]  
    
            pred_word = run_id_to_pred[int(sent.run_id.values[0])]
                                    
            pred_word_encodings = [pred_word]
            
            word_inputs.append([Sample(w) for w in word_encodings])
            pred_inputs.append([Sample(w) for w in pred_word_encodings])
            pos_inputs.append([Sample(pos) for pos in pos_tags_encodings])
            allPosEncoding.append(pos_tags_encodings) 
        return allPosEncoding 

def encodeXLnetInputs(sents,T):     
        word_inputs = []
        pred_inputs = []
        pos_inputs = []
        assert(all([len(set(sent.run_id.values)) == 1
                    for sent in sents]))        
         
        run_id_to_pred = dict([(int(sent.run_id.values[0]),
                                get_head_pred_word(sent))
                               for sent in sents])
        
        # Construct a mapping from running word index to pos
        word_id_to_pos = {}
        for sent in sents:
            indices = sent.index.values
            words = sent.word.values
   
   
            for index, word in zip(indices,
                                    spacy_ws(" ".join(words))):
                word_id_to_pos[index] = word.tag_  #tag_ get the tag of the word by spacy
        
  
        fixed_size_sents = get_fixed_size(sents)
        allPosEncoding=[] #19 jan 2020
        allWordsEncodingPartial=[]
        allWordsEncoding=[]
        allPredEncodingPartial=[]
        allPredEncoding=[]
        for sent in fixed_size_sents:

            assert(len(set(sent.run_id.values)) == 1)

            word_indices = sent.index.values
            sent_words = sent.word.values
       
            pos_tags_encodings = [p for (x,p) in nltk.pos_tag(sent_words)]
            word_encodings = [ w for w in sent_words]  
    
            pred_word = run_id_to_pred[int(sent.run_id.values[0])]
                                    
            pred_word_encodings = [pred_word]
            
            word_inputs.append([Sample(w) for w in word_encodings])
            pred_inputs.append([Sample(w) for w in pred_word_encodings])
            pos_inputs.append([Sample(pos) for pos in pos_tags_encodings])
            allPosEncoding.append(pos_tags_encodings) 
            allWordsEncodingPartial.append(word_encodings) #for Xlnet output is ['a','b'] need to make it space seperated using join
            allPredEncodingPartial.append(pred_word_encodings)
       
        if T=="train_xlnet_words" or T== "dev_xlnet_words" or T=="test_xlnet_words_RE" or T=="test_xlnet_words_" :  
          for sentence in allWordsEncodingPartial:
            x=[' '.join(sentence)]
            allWordsEncoding.append(x)
          return allWordsEncoding   
        elif  T=="train_xlnet_pred" or T== "dev_xlnet_pred"  or T=="test_xlnet_pred_RE" or T=="test_xlnet_pred_":  
          for sentence in allPredEncodingPartial:
            x=[' '.join(sentence)]
            allPredEncoding.append(x)
          return allPredEncoding 

#Block 4: Train File--> Adjust it according to your dataset and embeddings



In [ ]:
!python -m spacy download en
import spacy
spacy.load('en')
import pandas
import numpy as np
from keras.layers import Reshape, Flatten
from keras.models import load_model
from keras.layers.core import Dropout, Activation
import spacy
from symbols import SPACY_POS_TAGS
from collections import defaultdict
import nltk
from spacy_wrapper import spacy_whitespace_parser as spacy_ws 
from keras.layers.merge import concatenate, Concatenate,multiply
from keras.layers import TimeDistributed, Input, Bidirectional, Dense, Embedding, LSTM, Conv1D,Conv2D, Conv3D, GlobalMaxPooling1D, RepeatVector, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Masking, Activation, Input

#-------Train File-------------
#train_fn="add you file here"

dfE = pandas.read_csv(train_fn, sep= "\t",
                         header=0,
                         keep_default_na=False)

sents = get_sents_from_df(dfE)

train_textEI = dfE.groupby(dfE['word_id'].eq(0).cumsum())['word'].apply(lambda x: [' '.join(x)]).tolist()
train_wordsTokens=[]

for i in train_textEI:
  for j in i:
    train_wordsTokens.append(j.split())

train_wordsTokens_trunc=[]
for i in  train_wordsTokens:
  train_wordsTokens_trunc.append(i[:sent_maxlen])

train_predIE =  dfE.groupby(dfE['word_id'].eq(0).cumsum())['pred'].apply(lambda x: [' '.join(x)]).tolist()

train_predTokens=[]
for i in train_predIE:
  for j in i:
    train_predTokens.append(j.split())

train_predTokens_trunc=[]
for i in  train_wordsTokens:
  train_predTokens_trunc.append(i[:sent_maxlen])

Label_train=load_dataset(train_fn,"train")
X_train_elmo=load_dataset_encodeinputs(train_fn,"train")


# Block 4.2: Calling fucntions to get XLNet inputs+ shapes

**The snippet below is used for setting up the inputs for Embeddings**


In [ ]:
X_train_xlnet_Words=load_dataset_encodeinputs(train_fn,"train_xlnet_words")
mydiffLenSet=set() 
for sentence in X_train_xlnet_Words:
  mydiffLenSet.add(len(sentence[0].split()))
X_train_xlnet_pred=load_dataset_encodeinputs(train_fn,"train_xlnet_pred")
train_posIE =  load_dataset_encodeinputs(train_fn,"POS_train") 
train_posIE[0:5]
newTrain_posIE=[]
count=0
for sentence in train_posIE:
   count+=1
   newTrain_posIE.append(' '.join(sentence))

# **Block 5:** Dealing with padded sentences and then calling create_xlnet_embeddings

In [ ]:
for x in X_train_elmo['predicate_inputs']:
  x[x == '0'] =  x[0]
train_PredicateInjyXLNET= create_XLNET_embeddings("elmo", X_train_xlnet_pred, sent_maxlen)
for y in X_train_elmo['word_inputs']:
    y[y == '0'] = y[0]
train_WordsInjyXLNET= create_XLNET_embeddings("elmo", X_train_xlnet_Words, sent_maxlen)
for z in X_train_elmo['postags_inputs']:
    z[z == '0'] = z[0]
train_POSInjyXLNET= create_XLNET_embeddings("elmo", train_posIE, sent_maxlen)

# Block 6: Dev File 

In case you have a validation set; can be added as a part of test set by splitting

In [ ]:
#-------Dev File-------------
#dev_fn="add you dev data here"
Label_dev=load_dataset(dev_fn,"d")
X_dev_Xlnet_words=load_dataset_encodeinputs(dev_fn,"dev_xlnet_words")
X_dev_Xlnet_pred=load_dataset_encodeinputs(dev_fn,"dev_xlnet_pred")
X_dev_Xlnet_pos=load_dataset_encodeinputs(dev_fn,"dev_xlnet_pos")
dev_PredicateInjyXLNET= create_XLNET_embeddings("elmo", X_dev_Xlnet_pred, sent_maxlen)
dev_WordsInjyXLNET= create_XLNET_embeddings("elmo", X_dev_Xlnet_words, sent_maxlen)
dev_POSInjyXLNET= create_XLNET_embeddings("elmo", X_dev_Xlnet_pos, sent_maxlen)

# Block 7: The Model (stacking layers)

In [ ]:
!pip install keras-attention
!pip install keras-self-attention
import tensorflow.python.keras.layers
from tensorflow.keras.layers import Attention

In [ ]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K
from keras import layers
import pandas
import numpy as np
from keras.layers import Reshape, Flatten
from keras.models import load_model
from keras.layers.core import Dropout, Activation
from collections import defaultdict
from keras.layers.merge import concatenate, Concatenate,multiply
from keras.layers import TimeDistributed, Input, Bidirectional, Dense, Embedding, LSTM, Conv1D,Conv2D, Conv3D, GlobalMaxPooling1D, RepeatVector, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Masking, Activation, Input
from keras.layers import Conv1D, MaxPooling1D, Embedding, concatenate, Dropout, LSTM, GRU, Bidirectional, TimeDistributed, Masking

class AttentionLayer(Layer):

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  

        """ Computing energy outputs """
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [ ]:
emb_size=1024
def stack_latent_layers(n):
        return lambda x: stack(x, [lambda : Bidirectional(GRU(hidden_units,
                                                                    return_sequences = True))] * n )
def stack(x, layers):
        if not layers:
            return x 
        else:
            return layers[0]()(stack(x, layers[1:]))
def predict_classes():
        return lambda x: stack(x,
                                    [lambda : TimeDistributed(Dense(units = num_of_classes(),
                                                                    activation = "softmax"))] +
                                    [lambda : TimeDistributed(Dense(hidden_units,
                                                                    activation='relu'))] * 3)
  
inputs = [Input((sent_maxlen,emb_size), dtype='float32', name='word_inputs'),
            Input((sent_maxlen,emb_size), dtype='float32', name='predicate_inputs'),
          Input((sent_maxlen,emb_size), dtype='float32', name='POS_inputs'),]

embeddingsCon = [inputs[0], 
            inputs[1],
                inputs[2]]

con11 = keras.layers.concatenate(embeddingsCon)

latent_layers = stack_latent_layers(num_of_latent_layers)
bigru1=Bidirectional(LSTM(hidden_units, return_sequences = True))(con11)
bigru2=Bidirectional(LSTM(hidden_units, return_sequences = True))(bigru1)
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([bigru1, bigru2])
dropout = Dropout(0.1)(attn_out)
td=TimeDistributed(Dense(hidden_units,activation='relu'))(dropout)
outputI=TimeDistributed(Dense(units = num_of_classes(),activation = "softmax"))(td) 
model=Model(inputs, outputI)
model.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['categorical_accuracy'])
model.summary()
X=[train_WordsInjyXLNET,train_PredicateInjyXLNET,train_POSInjyXLNET]
X_dev=[dev_WordsInjyXLNET,dev_POSInjyXLNET,dev_POSInjyXLNET] 
model.fit(X, Label_train,
                       batch_size = 5,
                       epochs = 20,
                       validation_data = (X_dev, Label_dev),
           )

# **Block 8**: Testing-->load dataset for test set +  embedding + model.predict

In [ ]:
import pandas as pd
#test_fn="add your test file here"
Y=load_datasetTestRE(test_fn,"test_") 
X_test_Xlnet_words=load_dataset_encodeinputs(test_fn,"test_xlnet_words_") 
X_test_Xlnet_pred=load_dataset_encodeinputs(test_fn,"test_xlnet_pred_")
X_test_Xlnet_pos=load_dataset_encodeinputs(test_fn,"test_xlnet_pos_")
test_PredicateInjyXLNET= create_XLNET_embeddings("elmo", X_test_Xlnet_pred, sent_maxlen)
test_WordsInjyXLNET= create_XLNET_embeddings("elmo", X_test_Xlnet_words, sent_maxlen)
test_POSInjyXLNET= create_XLNET_embeddings("elmo", X_test_Xlnet_pos, sent_maxlen)

XTEST=[test_WordsInjyXLNET,test_PredicateInjyXLNET,test_POSInjyXLNET]
y = model.predict(XTEST)


# **Block 9 :** considalte labels and evaluation metrics 

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import logging
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib 
#labels=[Adapt based on your dataset labels]
myLabelsDict={k:v for k,v in enumerate(labels)}
for i,label in enumerate(y[10]):
  maxVal=np.max(label)
  index=np.where(label==maxVal)

def sample_labels( y, num_of_sents = 5, num_of_samples = 10,
                      num_of_classes = 3, start_index = 5, get_prob = True):
        classes = classes_()
        ret = []
        x=list(y)
        for sent in x[:num_of_sents]:
            cur = []
            for word in sent[start_index: start_index + num_of_samples]:
                sorted_prob = am(word)
                cur.append([(classes[ind], word[ind]) if get_prob 
                            else classes[ind]
                            for ind in sorted_prob[:num_of_classes]])
            ret.append(cur)
        return ret
def consolidate_labels(labels):
        x=map(consolidate_label , labels)
        return list(x)
      
def consolidate_label(label):
        return label.split("-")[0] if label.startswith("O") else label     
    
def transform_output_probs(y, get_prob = False):
        return np.array(sample_labels(y,
                                  num_of_sents = len(list(y)), # all sentences
                                  num_of_samples = max(list(map(len, y))), # all words
                                  num_of_classes = 1, # Only top probability
                                  start_index = 0, # all sentences  
                                  get_prob = get_prob, # Indicate whether to get only labels

               ))    
# Get most probable predictions and flatten
am = lambda myList: [i[0] for i in sorted(enumerate(myList), key=lambda x:x[1], reverse= True)]
Y1 = consolidate_labels(transform_output_probs(Y).flatten()) 
y2 = consolidate_labels(transform_output_probs(y).flatten()) 

logging.basicConfig(level = logging.DEBUG)

ret = []

eval_metrics = [
                ("F1 (micro)", lambda Y1, y2: metrics.f1_score(Y1, y2,average = 'micro')),
                ("Precision (micro)",lambda Y1, y2: metrics.precision_score(Y1, y2,average = 'micro')),
                ("Recall (micro)",lambda Y1, y2: metrics.recall_score(Y1, y2,average = 'micro')),
                ("Accuracy", metrics.accuracy_score),  
               ]

eval_metrics2 = [
                ("F1 (wieghted)", lambda Y1, y2: metrics.f1_score(Y1, y2,average = 'weighted')),
                ("Precision (wieghted)",lambda Y1, y2: metrics.precision_score(Y1, y2,average = 'weighted')),
                ("Recall (wieghted)",lambda Y1, y2: metrics.recall_score(Y1, y2,average = 'weighted')),
                ("Accuracy", metrics.accuracy_score),  
               ]
for (metric_name, metric_func) in eval_metrics:
        ret.append((metric_name, metric_func(Y1, y2)))
        logging.debug("calculating {}".format(ret[-1]))

for (metric_name, metric_func) in eval_metrics2:
        ret.append((metric_name, metric_func(Y1, y2)))
        logging.debug("calculating {}".format(ret[-1]))
      
for (metric_name, metric_val) in ret:
     logging.info("{}: {:.4f}".format(metric_name,
                                             metric_val))

def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig
from sklearn.metrics import confusion_matrix
#xx=[#labels here]
conM=confusion_matrix(Y1,y2)
figg=print_confusion_matrix(conM, xx, figsize = (10,7), fontsize=14)  
figg